In [12]:
import pandas as pd
import numpy as np

In [2]:
seq1 = "ATCGTAGC"
seq2 = "ATCGTACG"
seq3 = "GCTAGCTA"
seq4 = "TACGATCG"
rna1 = "AUGCUAGC"
rna2 = "AUGCUGCA"
rna3 = "GCUAGCUA"
rna4 = "UACGAUCG"
protein1 = "ACDEFGHIKLMNPQRSTVWY"
protein2 = "ACDFGHIKLMNPRSTVWY"
protein3 = "ACDEFGHLMNPQRSTVWX"
protein4 = "ABCDEFGHIJKLMN"


In [38]:
#нидлман-вунш
def needleman_wunsch(seq1, seq2, match=1, mismatch=-1, gap=-2):
    n = len(seq1)
    m = len(seq2)
    
    # создаем таблицу динамического программирования
    dp = [[0] * (m + 1) for _ in range(n + 1)]
    
    # заполняем первую строку и первый столбец (для пропусков)
    for i in range(n + 1):
        dp[i][0] = gap * i
    for j in range(m + 1):
        dp[0][j] = gap * j
    
    # заполняем таблицу динамического программирования
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            score = match if seq1[i - 1] == seq2[j - 1] else mismatch
            dp[i][j] = max(dp[i - 1][j - 1] + score,  # совпадение/несовпадение
                           dp[i - 1][j] + gap,       # пропуск в seq2
                           dp[i][j - 1] + gap)       # пропуск в seq1
    
    # Восстановление выравнивания
    aligned_seq1 = []
    aligned_seq2 = []
    
    i, j = n, m
    while i > 0 or j > 0:
        current_score = dp[i][j]
        
        # Если символы совпали или произошло несовпадение
        if i > 0 and j > 0 and current_score == dp[i - 1][j - 1] + ((match if seq1[i - 1] == seq2[j - 1] else mismatch)):
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append(seq2[j - 1])
            i -= 1
            j -= 1
        # пропуск в seq2
        elif i > 0 and current_score == dp[i - 1][j] + gap:
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append('-')
            i -= 1
        # пропуск в seq1
        elif j > 0 and current_score == dp[i][j - 1] + gap:
            aligned_seq1.append('-')
            aligned_seq2.append(seq2[j - 1])
            j -= 1
    
    # Обратим строки, так как мы строили выравнивание с конца
    aligned_seq1.reverse()
    aligned_seq2.reverse()
    
    # Возвращаем итоговый результат: балл и выровненные строки
    return dp[n][m], aligned_seq1, aligned_seq2

In [39]:
needleman_wunsch(seq1, seq2)

(4,
 ['A', 'T', 'C', 'G', 'T', 'A', 'G', 'C'],
 ['A', 'T', 'C', 'G', 'T', 'A', 'C', 'G'])

In [40]:
needleman_wunsch(seq1, seq3)

(-4,
 ['A', 'T', 'C', 'G', 'T', 'A', 'G', 'C', '-', '-'],
 ['-', 'G', 'C', '-', 'T', 'A', 'G', 'C', 'T', 'A'])

In [41]:
def smith_waterman(seq1, seq2, match=2, mismatch=-1, gap=-1):
    n = len(seq1)
    m = len(seq2)
    
    # создаем таблицу динамического программирования
    dp = [[0] * (m + 1) for _ in range(n + 1)]
    max_score = 0
    max_pos = (0, 0)
    
    # заполняем таблицу
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            score = match if seq1[i - 1] == seq2[j - 1] else mismatch
            dp[i][j] = max(0,
                           dp[i - 1][j - 1] + score,
                           dp[i - 1][j] + gap,
                           dp[i][j - 1] + gap)
            if dp[i][j] > max_score:
                max_score = dp[i][j]
                max_pos = (i, j)
    
    # dосстановление выравнивания
    aligned_seq1 = []
    aligned_seq2 = []
    
    i, j = max_pos
    while dp[i][j] > 0:
        current_score = dp[i][j]
        
        # если символы совпали или произошло несовпадение
        if i > 0 and j > 0 and current_score == dp[i - 1][j - 1] + ((match if seq1[i - 1] == seq2[j - 1] else mismatch)):
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append(seq2[j - 1])
            i -= 1
            j -= 1
        # если пропуск в seq2
        elif i > 0 and current_score == dp[i - 1][j] + gap:
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append('-')
            i -= 1
        # если пропуск в seq1
        elif j > 0 and current_score == dp[i][j - 1] + gap:
            aligned_seq1.append('-')
            aligned_seq2.append(seq2[j - 1])
            j -= 1
    
    # обратим строки
    aligned_seq1.reverse()
    aligned_seq2.reverse()
    
    return max_score, aligned_seq1, aligned_seq2

In [42]:
smith_waterman(seq1, seq2)

(13,
 ['A', 'T', 'C', 'G', 'T', 'A', '-', 'G'],
 ['A', 'T', 'C', 'G', 'T', 'A', 'C', 'G'])

In [43]:
smith_waterman(seq1, seq3)

(9, ['C', 'G', 'T', 'A', 'G', 'C'], ['C', '-', 'T', 'A', 'G', 'C'])